# Tupã: Multihead Encoders to Multihead Attention decoder.
Desc:

## Dependencies

In [1]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.df_models import *
from utils import *

## Defining variables

In [26]:
sequence_length_hour = 5  #hour
sequence_length_minute = 300 #minute
pred_length = 4 #hours
dict_values = ['dst_kyoto', 'kp_gfz']
device = get_default_device()

## Importing data and creating datasets

In [27]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = True)
l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean())

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000025132DD9E10>
Traceback (most recent call last):
  File "c:\Users\Innotronics\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "c:\Users\Innotronics\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
c:\Users\Innotronics\Desktop\SMFGF-SpaceApps\data\preprocessing.py:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mg1 = pd.concat(mg1_l

In [28]:
hour_kp_dataset = KpData(l1_sample_hour, kp, sequence_length_hour, pred_length, hour=True, sep = True)
minute_kp_dataset = KpData(l1_sample, kp, sequence_length_minute, pred_length, hour=False, sep = True)

In [29]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 256  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [30]:
#Test:5% training: 95%

test_size = round(0.05*len(minute_kp_dataset))

train_minute_kp, test_minute_kp = random_split(minute_kp_dataset , [len(minute_kp_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_minute_kp_dl = DataLoader(train_minute_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_minute_kp_dl = DeviceDataLoader(train_minute_kp_dl, device)
test_minute_kp_dl = DataLoader(test_minute_kp, batch_size*2, num_workers=4, pin_memory=True)
test_minute_kp_dl = DeviceDataLoader(test_minute_kp_dl, device)

## Hyperparameters

In [36]:
##hyperparameters
epochs = 20
max_lr = 1e-4
weigth_decay = 1e-6
grad_clip = 1e-3
opt_func = Adam
#opt_func = RMSprop

## Models training
**Architectures descriptions**

EncoderMultiheadAttentionLSTM(input, hidden, num_heads, architectures)

EncoderMultiheadAttentionLSTM(input, hidden, num_heads, architectures)

MultiHeaded2MultiheadAttentionLSTM(encoder_fc, encoder_mg,num_heads: list, architecture, output_size)


### Hour based

In [40]:
encoder_fc = EncoderMultiheadAttentionLSTM(9, 5, 3, (5,5))
encoder_mg = EncoderMultiheadAttentionLSTM(11, 5, 11, (5,5))
model  = MultiHead2SingleOUT(encoder_fc, encoder_mg,[3,11], (5,5), 1)

In [41]:
history_hour = []
history_hour += model.fit(epochs, max_lr,train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00001
	train_loss: 1.0489
	val_loss: 1.0235


### Saving the model

In [ ]:
torch.save(model.state_dict(), 'model_hour.pt')

### Minute based

In [ ]:
encoder_fc = EncoderMultiheadAttentionLSTM(9, 5, 3, (5,5))
encoder_mg = EncoderMultiheadAttentionLSTM(11, 5, 11, (5,5))
model  = MultiHead2SingleOUT(encoder_fc, encoder_mg,[3,11], (5,5), 1)

In [ ]:
history_minute = []
history_minute += model.fit(epochs, max_lr,train_minute_kp_dl, test_minute_kp_dl, weigth_decay, grad_clip, opt_func)

### Saving the model

In [ ]:
torch.save(model.state_dict(), 'model_minute.pt')